# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [3]:
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt467-project1
Project: mgmt467-project1 | Region: us-central1
Updated property [core/project].
mgmt467-project1


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

*We set `PROJECT_ID` and `REGION` at the top of the notebook so that all google cloud operations know which project and geographic location to use while executing the code. These variables ensure that resources like datasets, and models are created under the correct billing account and in a consistent region. If they aren’t set, API calls can fail due to missing project information, or resources might be created in the wrong region, leading to errors, higher costs, or access issues. Defining them once at the start helps maintain consistency, reproducibility, and smooth execution throughout the project.*

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [5]:
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

*Requiring strict 0600 permissions on API tokens like kaggle.json is very important for security. This setting allows only the file's owner to read and write to it, preventing unauthorized access by other users on the system. By limiting access, we significantly reduce the risk of exposing sensitive credentials that could be used to access or misuse your accounts and data. It's a standard practice to protect against potential data breaches and unauthorized actions.*

In [14]:
# Verification step: Show Kaggle CLI help output
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [7]:
!mkdir -p /content/data/raw
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
!unzip -o /content/data/*.zip -d /content/data/raw
# List CSV inventory
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 558MB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M A

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

*Keeping a clean file inventory with names and sizes is useful downstream for several reasons/purposes. It provides a clear record of the raw data available, which is essential for reproducibility and auditing. Knowing the file sizes helps in estimating storage needs and planning data processing tasks. It also makes it easier to locate specific files and verify that all expected data is present before proceeding with analysis or loading into other systems.*

In [15]:
import glob
import os

csv_files = glob.glob('/content/data/raw/*.csv')
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"

print("Found 6 CSV files:")
for csv_file in csv_files:
    print(os.path.basename(csv_file))

Found 6 CSV files:
movies.csv
watch_history.csv
search_logs.csv
users.csv
recommendation_logs.csv
reviews.csv


## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [9]:
import uuid, os
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name
!gcloud storage buckets create gs://$BUCKET_NAME --location=US
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
print("Bucket:", bucket_name)
# Verify contents
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating gs://mgmt467-netflix-faa7c31d/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-faa7c31d/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-faa7c31d/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-faa7c31d/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-faa7c31d/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-faa7c31d/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-faa7c31d/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-faa7c31d/netflix/watch_history.csv

Average throughput: 14.4MiB/s
Bucket: mgmt467-netflix-faa7c31d
gs://mgmt467-netflix-faa7c31d/netflix/README.md
gs://mgmt467-netflix-faa7c31d/netflix/movies.csv
gs://mgmt467-netflix-faa7c31d/netflix/recommendation_logs.csv
gs://mgmt467-n

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [16]:
# Verification step: List objects in the netflix/ prefix with sizes
import os
bucket_name = os.environ.get("BUCKET_NAME")
if bucket_name:
  !gcloud storage ls -l gs://$BUCKET_NAME/netflix/
else:
  print("BUCKET_NAME environment variable not set.")

      8002  2025-10-27T03:34:55Z  gs://mgmt467-netflix-faa7c31d/netflix/README.md
    115942  2025-10-27T03:34:55Z  gs://mgmt467-netflix-faa7c31d/netflix/movies.csv
   4695557  2025-10-27T03:34:55Z  gs://mgmt467-netflix-faa7c31d/netflix/recommendation_logs.csv
   1861942  2025-10-27T03:34:56Z  gs://mgmt467-netflix-faa7c31d/netflix/reviews.csv
   2250902  2025-10-27T03:34:56Z  gs://mgmt467-netflix-faa7c31d/netflix/search_logs.csv
   1606820  2025-10-27T03:34:55Z  gs://mgmt467-netflix-faa7c31d/netflix/users.csv
   9269425  2025-10-27T03:34:56Z  gs://mgmt467-netflix-faa7c31d/netflix/watch_history.csv
TOTAL: 7 objects, 19808590 bytes (18.89MiB)


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

*Two benefits of staging data in GCS compared to loading directly from local Colab are consistency and scalability. GCS provides a stable, central location for data that can be easily accessed by various services, unlike transient local Colab storage. Additionally, GCS is designed for scalability, handling large datasets efficiently, which is crucial for big data processing in BigQuery.*

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [10]:
DATASET="netflix"
# Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

BigQuery error in mk operation: Dataset 'mgmt467-project1:netflix' already
exists.
Dataset may already exist.


In [13]:
tables = {
  "users": "users.csv",
  "movies": "movies.csv",
  "watch_history": "watch_history.csv",
  "recommendation_logs": "recommendation_logs.csv",
  "search_logs": "search_logs.csv",
  "reviews": "reviews.csv",
}
import os
for tbl, fname in tables.items():
  src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
  print("Loading", tbl, "from", src)
  !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# Row counts
for tbl in tables.keys():
  # Use correct shell command formatting
  !bq query --nouse_legacy_sql "SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.{tbl}`"

Loading users from gs://mgmt467-netflix-faa7c31d/netflix/users.csv
Waiting on bqjob_r323266820e8be52_0000019a23bf6062_1 ... (1s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-faa7c31d/netflix/movies.csv
Waiting on bqjob_r652923419485a191_0000019a23bf7dd7_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-faa7c31d/netflix/watch_history.csv
Waiting on bqjob_r59285c6f23403e11_0000019a23bf9b61_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-faa7c31d/netflix/recommendation_logs.csv
Waiting on bqjob_r36e7615b95d85afd_0000019a23bfbd66_1 ... (1s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-faa7c31d/netflix/search_logs.csv
Waiting on bqjob_r686f5d3f48455020_0000019a23bfda01_1 ... (2s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-faa7c31d/netflix/reviews.csv
Waiting on bqjob_r183b58846911b048_0000019a23bffb9c_1 ... (1s) Current status: DONE   
/bin/bash: line

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


In [22]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])

query = f"""
SELECT 'users' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.reviews`
"""

results = client.query(query).result()

for row in results:
    print(f"{row.table_name}: {row.n} rows")




movies: 2080 rows
users: 20600 rows
reviews: 30900 rows
watch_history: 210000 rows
search_logs: 53000 rows
recommendation_logs: 104000 rows


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

*Autodetect is acceptable for initial data exploration or when the schema is simple and not likely to change, as it's quick and convenient. However, you should enforce explicit schemas when data quality and consistency are critical, or when dealing with complex nested structures or evolving data sources. Explicit schemas provide better control, data validation, and prevent unexpected data type conversions or errors during loads, ensuring data integrity for downstream analysis and applications.*

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [47]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])

query_missingness = f"""
WITH base AS (
  SELECT COUNT(*) AS n,
         COUNTIF(country IS NULL) AS miss_country,
         COUNTIF(subscription_plan IS NULL) AS miss_subscription_plan,
         COUNTIF(age IS NULL) AS miss_age
  FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.users`
)
SELECT n,
       ROUND(100*miss_country/n,2) AS pct_missing_country,
       ROUND(100*miss_subscription_plan/n,2) AS pct_missing_subscription_plan,
       ROUND(100*miss_age/n,2) AS pct_missing_age
FROM base;
"""

results = client.query(query_missingness).result()
for row in results:
    print(row)


Row((20600, 0.0, 0.0, 11.93), {'n': 0, 'pct_missing_country': 1, 'pct_missing_subscription_plan': 2, 'pct_missing_age': 3})


In [50]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])

query = f"""
SELECT country,
       COUNT(*) AS n,
       ROUND(100*COUNTIF(subscription_plan IS NULL)/COUNT(*),2) AS pct_missing_subscription_plan
FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.users`
GROUP BY country
ORDER BY pct_missing_subscription_plan DESC;
"""

results = client.query(query).result()

for row in results:
    print(row)



Row(('Canada', 6192, 0.0), {'country': 0, 'n': 1, 'pct_missing_subscription_plan': 2})
Row(('USA', 14408, 0.0), {'country': 0, 'n': 1, 'pct_missing_subscription_plan': 2})


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [54]:
# Verification query: Print the three missingness percentages
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

query = f"""
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(country IS NULL) miss_country,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `{project}.netflix.users`
)
SELECT ROUND(100*miss_country/n,2) AS pct_missing_country,
       ROUND(100*miss_plan/n,2) AS pct_missing_plan,
       ROUND(100*miss_age/n,2) AS pct_missing_age
FROM base;
"""

results = client.query(query).result()

for row in results:
    print(row)



Row((0.0, 0.0, 11.93), {'pct_missing_country': 0, 'pct_missing_plan': 1, 'pct_missing_age': 2})


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

*Based on the query results, the age_band column is the most missing, with about 11.93% of values being null. It's possible this is Missing At Random (MAR), where the missingness might be related to another unobserved user characteristic, or Missing Not At Random (MNAR) if users are less likely to provide their age for privacy reasons. region and plan_tier have no missing values, suggesting complete data for those columns.*

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [57]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

query = f"""
SELECT user_id, movie_id, watch_date, device_type, COUNT(*) AS dup_count
FROM `{project}.netflix.watch_history`
GROUP BY user_id, movie_id, watch_date, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20
"""

results = client.query(query).result()

for row in results:
    print(row)


Row(('user_00391', 'movie_0893', datetime.date(2024, 8, 26), 'Laptop', 8), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_03310', 'movie_0640', datetime.date(2024, 9, 8), 'Smart TV', 8), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_01383', 'movie_0015', datetime.date(2025, 4, 29), 'Desktop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_03021', 'movie_0602', datetime.date(2025, 2, 23), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_01182', 'movie_0794', datetime.date(2025, 7, 3), 'Desktop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_01807', 'movie_0921', datetime.date(2025, 1, 30), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_01870', 'movie_0844', datetime.date(2024, 6, 

In [61]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

# Replace 'watch_date' with your actual timestamp column (or event_ts)
timestamp_col = "watch_date"

query = f"""
CREATE OR REPLACE TABLE `{project}.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, {timestamp_col}, device_type
           ORDER BY {timestamp_col} DESC
         ) AS rk
  FROM `{project}.netflix.watch_history` h
)
WHERE rk = 1
"""

client.query(query).result()
print("✓ watch_history_dedup table created successfully")


✓ watch_history_dedup table created successfully


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [65]:
from google.cloud import bigquery
import os

# Set project
project = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project)

# SQL query: row counts for raw and dedup tables
sql = f"""
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count
FROM `{project}.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count
FROM `{project}.netflix.watch_history_dedup`
"""

# Run the query
results = client.query(sql).result()

# Print results
for row in results:
    print(row.table_name, row.row_count)


watch_history_dedup 100000
watch_history_raw 210000


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

*Duplicates can arise naturally from user actions (e.g., accidentally submitting a form twice) or be system-generated due to errors in data pipelines or logging. They corrupt labels and KPIs by artificially inflating counts, such as the number of views or interactions, leading to inaccurate analysis and potentially flawed business decisions or machine learning model training. Deduplication is crucial for ensuring data integrity and reliable metrics.*

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [70]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

sql = f"""
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `{project}.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `{project}.netflix.watch_history_dedup` h
CROSS JOIN bounds b;
"""

results = client.query(sql).result()

for row in results:
    print(row)


Row((3472, 100000, 3.47), {'outliers': 0, 'total': 1, 'pct_outliers': 2})


In [72]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

sql_create_robust = f"""
CREATE OR REPLACE TABLE `{project}.netflix.watch_history_robust` AS
SELECT *
FROM `{project}.netflix.watch_history_dedup`
WHERE watch_duration_minutes BETWEEN 0 AND 500;  -- example: keep only reasonable values
"""

# Run the query
client.query(sql_create_robust).result()
print("✓ watch_history_robust table created successfully")


✓ watch_history_robust table created successfully


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [74]:
# Query to show min/median/max before vs after capping
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

sql = f"""
SELECT
  'Before Capping' AS source,
  MIN(watch_duration_minutes) AS min_duration,
  APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_duration,
  MAX(watch_duration_minutes) AS max_duration
FROM `{project}.netflix.watch_history_dedup`
UNION ALL
SELECT
  'After Capping' AS source,
  MIN(watch_duration_minutes) AS min_duration,
  APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_duration,
  MAX(watch_duration_minutes) AS max_duration
FROM `{project}.netflix.watch_history_robust`;
"""

results = client.query(sql).result()

for row in results:
    print(row)


Row(('Before Capping', 0.2, 51.2, 799.3), {'source': 0, 'min_duration': 1, 'median_duration': 2, 'max_duration': 3})
Row(('After Capping', 0.2, 51.3, 499.5), {'source': 0, 'min_duration': 1, 'median_duration': 2, 'max_duration': 3})


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

*Capping might be harmful if the "outliers" represent genuine, important variations in the data that are critical for the analysis or model. Removing or altering them could lead to a loss of valuable information. Tree-based models like Decision Trees or Random Forests are generally less sensitive to outliers because they make decisions based on splitting data at certain thresholds rather than being influenced by the magnitude of extreme values like linear models are.*

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [76]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

sql = f"""
SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes > 8*60)/COUNT(*),2) AS pct
FROM `{project}.netflix.watch_history_robust`;
"""

results = client.query(sql).result()

for row in results:
    print(row)


Row((51, 87641, 0.06), {'sessions_over_8h': 0, 'total': 1, 'pct': 2})


In [79]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

sql = f"""
SELECT
  COUNTIF(age < 10 OR age > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct
FROM `{project}.netflix.users`;
"""

results = client.query(sql).result()

for row in results:
    print(row)


Row((358, 20600, 1.74), {'extreme_age_rows': 0, 'total': 1, 'pct': 2})


In [81]:
from google.cloud import bigquery
import os

client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])
project = os.environ['GOOGLE_CLOUD_PROJECT']

sql = f"""
SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 8*60) AS titles_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(duration_minutes < 15)/COUNT(*),2) AS pct_under_15m,
  ROUND(100*COUNTIF(duration_minutes > 8*60)/COUNT(*),2) AS pct_over_8h
FROM `{project}.netflix.movies`;
"""

results = client.query(sql).result()

for row in results:
    print(row)


Row((24, 22, 2080, 1.15, 1.06), {'titles_under_15m': 0, 'titles_over_8h': 1, 'total': 2, 'pct_under_15m': 3, 'pct_over_8h': 4})


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [83]:
from google.cloud import bigquery
import os

project = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project)

sql = f"""
SELECT 'flag_binge' AS flag_name,
       ROUND(100*COUNTIF(watch_duration_minutes > 8*60)/COUNT(*),2) AS pct_of_rows
FROM `{project}.netflix.watch_history_robust`

UNION ALL
SELECT 'flag_age_extreme' AS flag_name,
       ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct_of_rows
FROM `{project}.netflix.users`

UNION ALL
SELECT 'flag_duration_anomaly_under_15m' AS flag_name,
       ROUND(100*COUNTIF(duration_minutes < 15)/COUNT(*),2) AS pct_of_rows
FROM `{project}.netflix.movies`

UNION ALL
SELECT 'flag_duration_anomaly_over_8h' AS flag_name,
       ROUND(100*COUNTIF(duration_minutes > 8*60)/COUNT(*),2) AS pct_of_rows
FROM `{project}.netflix.movies`;
"""

results = client.query(sql).result()

for row in results:
    print(row)


Row(('flag_binge', 0.06), {'flag_name': 0, 'pct_of_rows': 1})
Row(('flag_age_extreme', 1.74), {'flag_name': 0, 'pct_of_rows': 1})
Row(('flag_duration_anomaly_under_15m', 1.15), {'flag_name': 0, 'pct_of_rows': 1})
Row(('flag_duration_anomaly_over_8h', 1.06), {'flag_name': 0, 'pct_of_rows': 1})


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

*Based on the results, flag_age_extreme is the most common anomaly flag at 1.74%. Whether to keep it as a feature depends on the modeling goal; if age is a strong predictor and these extremes are valid data points, flagging them could help the model. However, if they represent data entry errors, it might be better to treat them differently or exclude them.*

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


### Next Steps Checklist:

- [ ] Save this notebook to the team Drive.
- [ ] Export a `.sql` file with your DQ queries and save to repo.
- [ ] Push notebook + SQL to the team GitHub with a descriptive commit.
- [ ] Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.

## Project and Data Information

*   **Project ID:** mgmt467-project1
*   **Region:** us-central1
*   **GCS Bucket:** mgmt467-netflix-faa7c31d
*   **BigQuery Dataset:** mgmt467-project1.netflix

### Row Counts (from today's run)

*   **movies:** 2080 rows
*   **users:** 20600 rows
*   **reviews:** 30900 rows
*   **watch_history:** 210000 rows
*   **search_logs:** 53000 rows
*   **recommendation_logs:** 104000 rows

## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
